## Meat Stats Cold Storage

Setup

In [486]:
import pandas as pd
import functools as ft
TYPE_OF_MEAT = 'Type of Meat'

Load dataset

In [487]:
storage_df = pd.read_csv("../dataset/Meat_Stats_Cold_Storage.csv")
storage_df

,Date,Year,Month,Unit,Animal,Weight,Type_Of_Meat
0,Dec-2023,2023,12,Million Pounds,Beef,457.41,Red Meat
1,Nov-2023,2023,11,Million Pounds,Beef,445.67,Red Meat
2,Oct-2023,2023,10,Million Pounds,Beef,420.85,Red Meat
3,Sep-2023,2023,9,Million Pounds,Beef,395.40,Red Meat
4,Aug-2023,2023,8,Million Pounds,Beef,410.46,Red Meat
...,...,...,...,...,...,...,...
3931,May-1983,1983,5,Million Pounds,Frozen Eggs,NaN,Poultry
3932,Apr-1983,1983,4,Million Pounds,Frozen Eggs,NaN,Poultry
3933,Mar-1983,1983,3,Million Pounds,Frozen Eggs,NaN,Poultry
3934,Feb-1983,1983,2,Million Pounds,Frozen Eggs,NaN,Poultry


In [488]:
storage_df['Year'] = storage_df['Year'].astype(str)
storage_df['Month'] = storage_df['Month'].astype(str)

storage_df['DateTime'] = storage_df['Year'] + '-' + storage_df['Month'].str.rjust(2, '0')

storage_df['Weight'] = storage_df['Weight'] * 1e6

storage_df = storage_df.drop(columns=['Date', 'Unit', 'Year', 'Month'])

storage_df = storage_df[~storage_df['Weight'].isnull()]

storage_df = storage_df[['DateTime', 'Animal', 'Weight', 'Type_Of_Meat']]
storage_df = storage_df.rename(columns={'Type_Of_Meat': TYPE_OF_MEAT})

In [489]:
storage_df.dtypes

DateTime         object
Animal           object
Weight          float64
Type of Meat     object
dtype: object

# Meat Stats Meat Production

In [490]:
production_df = pd.read_csv("../dataset/Meat_Stats_Meat_Production.csv")

In [491]:
production_df['Year'] = production_df['Year'].astype(str)
production_df['Month'] = production_df['Month'].astype(str)

production_df['DateTime'] = production_df['Year'] + '-' + production_df['Month'].str.rjust(2, '0')

production_df['Production'] = production_df['Production'].str.replace(',', '').astype(float)
production_df['Production'] = production_df['Production'] * 1e6
production_df = production_df.drop(columns=['Date', 'Unit', 'Year', 'Month'])

production_df = production_df[~production_df['Production'].isnull()]

production_df = production_df[['DateTime', 'Animal', 'Production', 'Type of Meat', 'Commercial or Federally Inspected']]
production_df = production_df.rename(columns={'Type of Meat': TYPE_OF_MEAT})
production_df

,DateTime,Animal,Production,Type of Meat,Commercial or Federally Inspected
0,2023-12,Beef,2.189800e+09,Red Meat,Commercial
1,2023-11,Beef,2.280900e+09,Red Meat,Commercial
2,2023-10,Beef,2.340900e+09,Red Meat,Commercial
3,2023-09,Beef,2.149400e+09,Red Meat,Commercial
4,2023-08,Beef,2.363500e+09,Red Meat,Commercial
...,...,...,...,...,...
13121,1960-06,Turkey,2.710000e+07,Poultry,Federally Inspected
13122,1960-05,Turkey,1.620000e+07,Poultry,Federally Inspected
13123,1960-04,Turkey,1.340000e+07,Poultry,Federally Inspected
13124,1960-03,Turkey,1.400000e+07,Poultry,Federally Inspected


In [492]:
production_df = production_df[production_df['Commercial or Federally Inspected'] == 'Federally Inspected']
production_df

animals = production_df['Animal'].unique()
print(animals)

animal_df = {animal: production_df[production_df['Animal'] == animal] for animal in animals}
for animal in animals:
    animal_df[animal] = animal_df[animal].rename(columns={'Production': animal.capitalize().replace(" ", "_")})
    animal_df[animal] = animal_df[animal].drop(columns=['Animal', 'Type of Meat', 'Commercial or Federally Inspected'])

['Beef' 'Veal' 'Pork' 'Lamb and Mutton' 'Broilers' 'Other Chicken'
 'Turkey']


In [493]:
df_final = ft.reduce(lambda left, right: pd.merge(left, right, on='DateTime'), animal_df.values())
df_final.to_csv("../udataset/meat_production.csv")
df_final

,DateTime,Beef,Veal,Pork,Lamb_and_mutton,Broilers,Other_chicken,Turkey
0,2023-12,2.158900e+09,3700000.0,2.332400e+09,9500000.0,3.615000e+09,41500000.0,449200000.0
1,2023-11,2.247300e+09,4100000.0,2.377400e+09,10100000.0,3.817300e+09,44500000.0,500300000.0
2,2023-10,2.300800e+09,4000000.0,2.406200e+09,9800000.0,4.177300e+09,50600000.0,431200000.0
3,2023-09,2.114600e+09,3500000.0,2.175500e+09,8600000.0,3.805500e+09,48400000.0,489200000.0
4,2023-08,2.329500e+09,4100000.0,2.282400e+09,8900000.0,4.156700e+09,51100000.0,433500000.0
...,...,...,...,...,...,...,...,...
271,2001-05,2.265000e+09,15000000.0,1.535000e+09,16000000.0,2.835600e+09,45600000.0,428800000.0
272,2001-04,1.910000e+09,15000000.0,1.514000e+09,19000000.0,2.515700e+09,42200000.0,462000000.0
273,2001-03,2.065000e+09,16000000.0,1.606000e+09,23000000.0,2.588600e+09,44400000.0,409300000.0
274,2001-02,1.852000e+09,15000000.0,1.467000e+09,17000000.0,2.322200e+09,39700000.0,461200000.0


# Meat Stats Slaughter Counts

In [494]:
slaughter_df = pd.read_csv("../dataset/Meat_Stats_Slaughter_Counts.csv")
slaughter_df

,Date,Year,Month,Unit,Animal,Count,Commercial_Or_Federally_Inspected,Type_Of_Meat
0,Dec-2023,2023,12,"1,000 Head",Cattle,"2,593.40",Commercial,Red Meat
1,Nov-2023,2023,11,"1,000 Head",Cattle,"2,729.90",Commercial,Red Meat
2,Oct-2023,2023,10,"1,000 Head",Cattle,"2,826.00",Commercial,Red Meat
3,Sep-2023,2023,9,"1,000 Head",Cattle,"2,610.60",Commercial,Red Meat
4,Aug-2023,2023,8,"1,000 Head",Cattle,"2,890.00",Commercial,Red Meat
...,...,...,...,...,...,...,...,...
15278,Apr-1983,1983,4,"1,000 Head",Turkeys,"10,563.00",Federally Inspected,Poultry
15279,Mar-1983,1983,3,"1,000 Head",Turkeys,"11,979.00",Federally Inspected,Poultry
15280,Feb-1983,1983,2,"1,000 Head",Turkeys,"8,477.00",Federally Inspected,Poultry
15281,Jan-1983,1983,1,"1,000 Head",Turkeys,"8,532.00",Federally Inspected,Poultry


In [495]:
slaughter_df['Year'] = slaughter_df['Year'].astype(str)
slaughter_df['Month'] = slaughter_df['Month'].astype(str)

slaughter_df['DateTime'] = slaughter_df['Year'] + '-' + slaughter_df['Month'].str.rjust(2, '0')

In [496]:
slaughter_df['Count'] = slaughter_df['Count'].str.replace(',', '').astype(float)
slaughter_df['Count'] = slaughter_df['Count'] * 1e3
slaughter_df = slaughter_df.drop(columns=['Date', 'Unit', 'Year', 'Month'])

In [497]:
# slaughter_df = slaughter_df[['DateTime', 'Animal', 'Production', 'Commercial or Federally Inspected', 'Type_Of_Meat']]
# slaughter_df = slaughter_df.rename(columns={'Type_Of_Meat': TYPE_OF_MEAT})
# slaughter_df

KeyError: "['Production', 'Commercial or Federally Inspected'] not in index"